1. 用avg on training set作为testing set的param value值
2. 然后计算testing set的metric

In [157]:
import torch
import numpy as np
import random
import pandas as pd
import os
from tqdm import tqdm
from metric_for_GTs_func import *


ARTIFICIAL = True
noise_pct = 0.05

if ARTIFICIAL:
    target_path = "../data/artificial_targets_v2_" + "noise=" + str(noise_pct)
    # SA 的 params
    params_opitim_path_SA = "../data/SA_PT/params_opitim_artificial_v2_noise_"+str(noise_pct)+".csv"
else:
    target_path = "../data/targets_all"
    # SA 的 params
    params_opitim_path_SA = "../data/SA_PT/params_opitim_delta_T.csv"
# Small dataset
small_settings_NN_path = r"../data/small_settings_NN.csv"
# Large data
large_settings_NN_path = r'E:\DATA\large_dta\large_settings_NN.csv'


In [158]:
# target_path里有全部的target data地址
target_all_path = os.listdir(target_path)

params_opitim_SA = pd.read_csv(params_opitim_path_SA,encoding="utf-8")

data_key_small = pd.read_csv(small_settings_NN_path,encoding="utf-8")
data_key_large = pd.read_csv(large_settings_NN_path,encoding="utf-8")

data_key = pd.concat([data_key_small,data_key_large],axis=0,ignore_index=True)

In [159]:
def setup_seed(seed):
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    np.random.seed(seed)
    random.seed(seed)
    torch.backends.cudnn.deterministic = True

train_pct=0.7
vali_pct=0.1

In [160]:
seed = 407       # 3,31,204,223,407 (307)
setup_seed(seed)

DATA_len = len(target_all_path)
shuffled_indices = np.random.permutation(DATA_len)
DATA_len = len(shuffled_indices)


In [161]:
params_opitim_SA.head()

,key_idx,alpha,delta,labda,initial_loss,final_loss,avg_loss
0,0,0.061227,1,4.007592,1048.245937,971.189029,3.034966
1,1,0.047061,1,7.664214,939.993322,804.370159,2.654687
2,2,0.080019,1,4.080966,1710.033256,1506.151280,2.789169
3,3,0.097774,1,5.198637,737.680055,589.685310,2.330772
4,4,0.021540,1,7.627588,1086.205018,1041.966627,3.020193


- 计算avg param values

In [162]:
train_idx = shuffled_indices[:int(train_pct * DATA_len)]
params_train = params_opitim_SA.iloc[train_idx,:].copy()

params_avg = params_train.reset_index().mean()

- 计算testing上的metric

In [163]:

tmp = int((train_pct + vali_pct) * DATA_len)
val_idx = shuffled_indices[int(train_pct * DATA_len):tmp]
test_idx = shuffled_indices[tmp:]
NLL_metric = []

alpha = params_avg.alpha
labda = params_avg.labda
alpha,labda

(0.03766712134864847, 4.08987890126484)

In [164]:
for i in test_idx:
        # Get target data
        target_path_i_path = os.path.join(target_path, target_all_path[i])
        target_df = pd.read_csv(target_path_i_path,encoding="utf-8")
        target = list(target_df.iloc[:,0])

        # Get the auction setting
        settings_df = data_key.iloc[i,:]

        # GT-2: SA-AVG
        NLL_metric.append(GT_2_uniq(settings_df,target,alpha,labda))


In [165]:
len(NLL_metric)

256

In [166]:
print("seed=",seed)
np.mean(NLL_metric)

seed= 407


8.193912674121918